# Classifying Data with VGG16 model
- VGG16 won 2014 imagenet 

In [1]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense,Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

Using TensorFlow backend.


## Getting Data

In [2]:
train_path = 'cats-and-dogs/train'
valid_path = 'cats-and-dogs/valid'
test_path  = 'cats-and-dogs/test'

train_batches = ImageDataGenerator().flow_from_directory(directory=train_path,target_size = (224,224),
                                                         classes  = ['dog','cat'],batch_size=10)

valid_batches = ImageDataGenerator().flow_from_directory(directory=valid_path,target_size = (224,224),
                                                         classes  = ['dog','cat'],batch_size=4)

test_batches = ImageDataGenerator().flow_from_directory(directory=test_path,target_size = (224,224),
                                                         classes  = ['dog','cat'])

Found 48 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


### Importing VGG16 Model

In [3]:
vgg16_model = keras.applications.vgg16.VGG16()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



The Original Trained VGG16 model, along with saved weights and hyperparameters has been downloaded

In [4]:
type(vgg16_model)

keras.engine.training.Model

In [5]:
vgg16_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

- The last layer ; predictions layer is 1000 due to the presence of 1000 category on imagenet
- We will have to change it to 2 as we only need to see if our image is a cat or dog
- **We will now have to create this Functional Model to a Sequential Model in order to change the last layer**
- This we first create a new **Sequential** model and add all the layers of vgg16 leaving the last layer

In [6]:
model = Sequential()
for layer in vgg16_model.layers[:-1]:
    model.add(layer)

- Now we have relpicated all the layers , leaving out the output layer
- Now we will iterate over the layers and set them to non-trainable
- Actually along with the layers we had also added the weights of the vgg16_model to our model.
- Setting them to non-trainable freezes the weights and other trainable parameters
- So when we train the model we train only the last layer

In [7]:
for layer in model.layers:
    layer.trainable = False

**Adding our output layer**

In [8]:
model.add(Dense(2,activation='softmax'))

In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

# Training

Compile Model

In [10]:
model.compile(optimizer=Adam(learning_rate=0.0001),loss = 'categorical_crossentropy',metrics=['accuracy'])

Training

In [11]:
model.fit_generator(generator = train_batches,validation_data=valid_batches,epochs=10,verbose=2)


Epoch 1/10
 - 8s - loss: 1.6129 - accuracy: 0.5000 - val_loss: 0.4800 - val_accuracy: 0.6500
Epoch 2/10
 - 8s - loss: 1.1661 - accuracy: 0.5833 - val_loss: 0.4802 - val_accuracy: 0.6500
Epoch 3/10
 - 8s - loss: 0.8149 - accuracy: 0.6458 - val_loss: 0.0438 - val_accuracy: 0.7000
Epoch 4/10
 - 8s - loss: 0.5724 - accuracy: 0.7917 - val_loss: 0.0317 - val_accuracy: 0.8000
Epoch 5/10
 - 9s - loss: 0.4199 - accuracy: 0.8125 - val_loss: 0.0248 - val_accuracy: 0.8500
Epoch 6/10
 - 8s - loss: 0.3065 - accuracy: 0.8750 - val_loss: 0.6641 - val_accuracy: 0.8500
Epoch 7/10
 - 8s - loss: 0.2164 - accuracy: 0.9167 - val_loss: 0.3273 - val_accuracy: 0.8500
Epoch 8/10
 - 9s - loss: 0.1746 - accuracy: 0.9583 - val_loss: 0.1901 - val_accuracy: 0.9000
Epoch 9/10
 - 9s - loss: 0.1367 - accuracy: 0.9583 - val_loss: 0.1923 - val_accuracy: 0.9500
Epoch 10/10
 - 9s - loss: 0.1067 - accuracy: 0.9792 - val_loss: 0.1403 - val_accuracy: 0.9500


In [12]:
test_imgs,test_labels = next(test_batches)

In [13]:
test_labels

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [14]:
test_labels = test_labels[:,0]

In [15]:
predictions = model.predict_generator(generator = test_batches,verbose=1)

1/1 [==============================] - 2s 2s/step


In [16]:
print(predictions)

[[9.9215007e-01 7.8498730e-03]
 [9.9385417e-01 6.1458140e-03]
 [9.4894153e-01 5.1058520e-02]
 [3.1384546e-03 9.9686158e-01]
 [1.4813584e-03 9.9851865e-01]
 [6.6667610e-01 3.3332390e-01]
 [3.8620863e-02 9.6137917e-01]
 [9.4975722e-01 5.0242752e-02]
 [3.4022596e-02 9.6597743e-01]
 [9.8394379e-02 9.0160561e-01]
 [9.2373282e-01 7.6267138e-02]
 [6.5547407e-02 9.3445259e-01]
 [1.3095682e-03 9.9869043e-01]
 [1.2481202e-04 9.9987519e-01]
 [2.6432052e-01 7.3567945e-01]
 [4.0533432e-01 5.9466565e-01]
 [4.2305328e-02 9.5769471e-01]
 [1.1229761e-01 8.8770235e-01]
 [9.8519355e-01 1.4806469e-02]
 [9.3459022e-01 6.5409817e-02]]


In [17]:
for k in predictions:
    if(k[0]>k[1]):
        k[0]=1
        k[1]=0
    else:
        k[0]=0
        k[1]=1

In [18]:
predictions = predictions[:,0]
predictions

array([1., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 1., 1.], dtype=float32)

In [19]:
c=0
for i in range(len(predictions)):
    if(predictions[i]==test_labels[i]):
        c+=1
print('Accuracy : ', c*100/len(predictions))

Accuracy :  90.0


In [20]:
CM = confusion_matrix(y_true=test_labels,y_pred = predictions)
print(CM)

[[10  0]
 [ 2  8]]
